# Thêm GROQ_API_KEY và môi trường

In [2]:
import os
os.environ['GROQ_API_KEY'] = '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'

# Tải các thư viện

In [10]:
!pip install -q groq
!pip install -q langchain
!pip install -q langchain_groq
!pip install -q langchain_huggingface
!pip install -q datasets
!pip install -q chromadb
!pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00


# Load datasets

In [6]:
from datasets import load_dataset
data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

In [7]:
data = data.to_pandas()
data.head(10)

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
5,prevention,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...
6,information,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...
7,susceptibility,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
8,exams and tests,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ..."
9,treatment,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...


In [8]:
# Chọn  100 mẫu dữ liệu
#uncoment this line if you want to limit the size of the data.
data = data[0:100]

In [11]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

In [12]:
df_loader = DataFrameLoader(data, page_content_column="Answer")
df_document = df_loader.load()
display(df_document[:2])

[Document(metadata={'qtype': 'susceptibility', 'Question': 'Who is at risk for Lymphocytic Choriomeningitis (LCM)? ?'}, page_content='LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.'),
 Document(metadata={'qtype': 'symptoms', 'Question': 'What are the symptoms of Lymphocytic Choriomeningitis (LCM) ?'}, page_content='LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. \n                \nFor infected persons who do become ill, onset of sympt

# Split text

In [13]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1250,
                                      separator="\n",
                                      chunk_overlap=100)
texts = text_splitter.split_documents(df_document)


In [14]:
first_doc = texts[1]
print(first_doc.page_content)

LCMV is most commonly recognized as causing neurological disease, as its name implies, though infection without symptoms or mild febrile illnesses are more common clinical manifestations. 
                
For infected persons who do become ill, onset of symptoms usually occurs 8-13 days after exposure to the virus as part of a biphasic febrile illness. This initial phase, which may last as long as a week, typically begins with any or all of the following symptoms: fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. Other symptoms appearing less frequently include sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain.


# Khởi tạo mô hình Embedding

In [15]:
from langchain_huggingface import HuggingFaceEmbeddings
embed = HuggingFaceEmbeddings()



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Tích hợp Embedding HuggingFace vào ChromaDB

In [16]:
directory_cdb = '/content/drive/MyDrive/chromadb'
chroma_db = Chroma.from_documents(
    df_document, embed, persist_directory=directory_cdb
)

In [20]:
from langchain_groq import ChatGroq
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA


client = ChatGroq(
    api_key = os.environ.get('GROQ_API_KEY'),
    model= 'mixtral-8x7b-32768'
)

# Tạo history cho đoạn chat
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

# Tạo một mô hình RAG cơ bản với langchain
qa = RetrievalQA.from_chain_type(
    llm=client,
    chain_type="stuff",
    retriever=chroma_db.as_retriever()
)
# Lưu ý khi sử dụng nhanh mô hình RAG do langchain cung cấp thì nên sử dụng các model
# LLM mà langchain hỗ trợ

# Test thử mô hình RAG cơ bản

In [21]:
qa.run("What is the main symptom of LCM?")

<ipython-input-21-e02068ce0ffd>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run("What is the main symptom of LCM?")


'The main symptom of LCM (Lymphocytic Chorio meningitis) is a biphasic febrile illness. The first phase, which may last up to a week, includes fever, malaise, lack of appetite, muscle aches, headache, nausea, and vomiting. Other symptoms appearing less frequently include sore throat, cough, joint pain, chest pain, testicular pain, and parotid (salivary gland) pain. A second phase of illness may occur following a few days of recovery, and symptoms may consist of meningitis, encephalitis, or meningoencephalitis. LCMV has also been known to cause acute hydrocephalus, myelitis, and an association between LCMV infection and myocarditis has been suggested.'

# Khởi tạo Agents

Khởi tạo các Tool để phục vụ cho Agents. Bao gồm: Tên tool, công cụ hỗ trợ ở đây dùng `qa.run` là 1 mô hình RAG được tạo phía trên sẽ cho phép Agents thực hiện retrieval nếu cân thiết và mô tả về việc khi nào Agents cần sử dụng tool đó.

In [22]:
from langchain.agents import Tool

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Medical KB',
        func=qa.run,
        description=(
            """use this tool when answering medical knowledge queries to get
            more information about the topic"""
        )
    )
]


# Định nghĩa Agents

In [23]:
from langchain.agents import create_react_agent
from langchain import hub

# Khởi tạo các prompt được langchain cung cấp
prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(
    tools=tools,
    llm=client,
    prompt=prompt,
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# Tạo Agents Executor để thực thi tác vụ

In [24]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               handle_parsing_errors=True
                               )

# Thử nghiệm Agents

In [26]:
agent_executor.invoke({"input": "Give me the area of square of 2x2"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: The area of a square with side length 2x2 is (2x2) x (2x2) = 4x4 = 16 square units.

> Finished chain.


{'input': 'Give me the area of square of 2x2',
 'chat_history': [],
 'output': 'The area of a square with side length 2x2 is (2x2) x (2x2) = 4x4 = 16 square units.'}

Có thể nhận xét về Agents đã xác định được câu hỏi không liên quan đến bệnh. Nên nó không sử dụng tool

In [27]:
agent_executor.invoke({"input": "Do you know who is Clark Kent?"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Yes, I can provide information on that. Clark Kent is a fictional character in the DC Universe of comic books. He is the secret identity of Superman, a superhero who was born on the planet Krypton and was raised on Earth. Clark Kent works as a journalist for the Daily Planet in Metropolis, and in this role, he often investigates stories and covers events related to Superman's activities. In his superhero persona, Superman has a variety of superhuman abilities, including strength, speed, invulnerability, flight, and heat vision. He uses these powers to protect the people of Earth from various threats and to uphold justice.

> Finished chain.


{'input': 'Do you know who is Clark Kent?',
 'chat_history': [HumanMessage(content='Give me the area of square of 2x2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The area of a square with side length 2x2 is (2x2) x (2x2) = 4x4 = 16 square units.', additional_kwargs={}, response_metadata={})],
 'output': "Yes, I can provide information on that. Clark Kent is a fictional character in the DC Universe of comic books. He is the secret identity of Superman, a superhero who was born on the planet Krypton and was raised on Earth. Clark Kent works as a journalist for the Daily Planet in Metropolis, and in this role, he often investigates stories and covers events related to Superman's activities. In his superhero persona, Superman has a variety of superhuman abilities, including strength, speed, invulnerability, flight, and heat vision. He uses these powers to protect the people of Earth from various threats and to uphold justice."}

In [28]:
agent_executor.memory.clear()

In [29]:
agent_executor.invoke({"input": """I have a patient that can have Botulism,
how can I confirm the diagnosis?"""})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Medical KB
Action Input: "How to confirm Botulism diagnosis"
To confirm a Botulism diagnosis, special tests are usually needed since other diseases can appear similar to Botulism. These tests can include a brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), and a tensilon test for myasthenia gravis. Additionally, tests for Botulinum toxin and for bacteria that cause Botulism can be performed at some state health department laboratories and at CDC. If you suspect a case of Botulism, it should be reported to your state health department immediately.Do I need to use a tool? No
Final Answer: To confirm a diagnosis of Botulism, special tests are typically required as other diseases can have similar symptoms. These tests may include a brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), and a tensilon test for myasthenia gravis. Furtherm

{'input': 'I have a patient that can have Botulism,\nhow can I confirm the diagnosis?',
 'chat_history': [],
 'output': 'To confirm a diagnosis of Botulism, special tests are typically required as other diseases can have similar symptoms. These tests may include a brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), and a tensilon test for myasthenia gravis. Furthermore, tests for Botulinum toxin and for bacteria that cause Botulism can be conducted at certain state health department laboratories and at the CDC. If you suspect a case of Botulism, it is crucial to report it to your state health department promptly.'}

In [30]:
agent_executor.invoke({"input": "Is this an important illness?"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No.
Final Answer: Yes, Botulism is an important illness because it can be life-threatening if not promptly diagnosed and treated. It is a rare but serious condition caused by a toxin that attacks the body's nerves, causing muscle paralysis. If left untreated, it can lead to respiratory failure and death. Therefore, if you suspect a case of Botulism, it is crucial to report it to your state health department promptly and seek immediate medical attention.

> Finished chain.


{'input': 'Is this an important illness?',
 'chat_history': [HumanMessage(content='I have a patient that can have Botulism,\nhow can I confirm the diagnosis?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='To confirm a diagnosis of Botulism, special tests are typically required as other diseases can have similar symptoms. These tests may include a brain scan, spinal fluid examination, nerve conduction test (electromyography, or EMG), and a tensilon test for myasthenia gravis. Furthermore, tests for Botulinum toxin and for bacteria that cause Botulism can be conducted at certain state health department laboratories and at the CDC. If you suspect a case of Botulism, it is crucial to report it to your state health department promptly.', additional_kwargs={}, response_metadata={})],
 'output': "Yes, Botulism is an important illness because it can be life-threatening if not promptly diagnosed and treated. It is a rare but serious condition caused by a toxin that attacks 

`Nhận xét`: Ở câu hỏi đầu tiên Agents xác định được cần phải gọi tool. Ở câu trả lời thứ 2, Agents xác định được không cần phải gọi tool